In [21]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import scipy.ndimage as sn

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull
from skimage import measure
import warnings

In [22]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [23]:
#name_project = 'project/output_FJSJ_17-01/'               
#name_project = 'project_repartition_v3.0/output_repar_v9.1_02-01/'     
#name_project = 'project_repartition_v4.0/output_repar_v9.2_01-01/'
#name_project = 'project_voro_v1.0/output_voro_v9.3_01-01/'             
name_project = 'project_voro_v1.0/output_voro_v9.3_test/'      
name_project

'project_voro_v1.0/output_voro_v9.3_test/'

In [24]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_voro_v1.0/output_voro_v9.3_test/


In [25]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [26]:
key_subworks = info_basic['key_subworks']
#key_subworks = info_basic['key_subworks_repick']
key_subworks

['vor0_0',
 'vor0_1',
 'vor0_2',
 'vor0_3',
 'vor0_4',
 'vor0_5',
 'vor0_6',
 'vor0_7',
 'vor0_8',
 'vor0_9',
 'vor0_10',
 'vor0_11',
 'vor0_12',
 'vor0_13',
 'vor0_14',
 'vor0_15',
 'vor0_16',
 'vor0_17',
 'vor0_18',
 'vor0_19',
 'vor0_20',
 'vor0_21',
 'vor0_22',
 'vor0_23',
 'vor0_24',
 'vor0_25',
 'vor0_26',
 'vor0_27',
 'vor0_28',
 'vor0_29',
 'vor1_0',
 'vor1_1',
 'vor1_2',
 'vor1_3',
 'vor1_4',
 'vor1_5',
 'vor1_6',
 'vor1_7',
 'vor1_8',
 'vor1_9',
 'vor1_10',
 'vor1_11',
 'vor1_12',
 'vor1_13',
 'vor1_14',
 'vor1_15',
 'vor1_16',
 'vor1_17',
 'vor1_18',
 'vor1_19',
 'vor1_20',
 'vor1_21',
 'vor1_22',
 'vor1_23',
 'vor1_24',
 'vor2_0',
 'vor2_1',
 'vor2_2',
 'vor2_3',
 'vor2_4',
 'vor2_5',
 'vor2_6',
 'vor2_7',
 'vor2_8',
 'vor2_9',
 'vor2_10',
 'vor2_11',
 'vor2_12',
 'vor2_13',
 'vor2_14',
 'vor2_15',
 'vor2_16',
 'vor2_17',
 'vor2_18',
 'vor2_19',
 'vor2_20',
 'vor2_21',
 'vor2_22',
 'vor2_23',
 'vor3_0',
 'vor3_1',
 'vor3_2',
 'vor3_3',
 'vor3_4',
 'vor3_5',
 'vor3_6',
 'vor

In [27]:
dir_image = dir_project+info_basic['dir_image'] + 'autopick/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_ds = dir_project+info_basic['dir_ds']
print('dir_image: ', dir_image)
print('dir_ds: ', dir_ds)

dir_image:  /shdisk/rem2/Harmon/F-J/San/project_voro_v1.0/output_voro_v9.3_test/image_v9.3_test/autopick/
dir_ds:  /shdisk/rem2/Harmon/F-J/San/project_voro_v1.0/output_voro_v9.3_test/ds_v9.3_test/


In [28]:
dir_disp =  'autopick'+info_basic['tag']+'/'
info_basic['dir_disp'] = dir_disp
dir_disp = dir_project+dir_disp
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)
print('dir_disp: ', dir_disp)

dir_disp:  /shdisk/rem2/Harmon/F-J/San/project_voro_v1.0/output_voro_v9.3_test/autopickv9.3_test/


In [29]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [30]:
ds_all = os.listdir(dir_ds)
ds_all

['vor9_16.h5',
 'vor5_21.h5',
 'vor0_24.h5',
 'vor2_21.h5',
 'vor1_22.h5',
 'vor5_14.h5',
 'vor5_18.h5',
 'vor4_16.h5',
 'vor2_0.h5',
 'vor1_4.h5',
 'vor7_1.h5',
 'vor4_6.h5',
 'vor7_10.h5',
 'vor6_15.h5',
 'vor8_25.h5',
 'vor7_23.h5',
 'vor6_4.h5',
 'vor3_11.h5',
 'vor4_17.h5',
 'vor9_12.h5',
 'vor8_8.h5',
 'vor1_10.h5',
 'vor9_19.h5',
 'vor7_15.h5',
 'vor4_10.h5',
 'vor3_9.h5',
 'vor7_24.h5',
 'vor5_13.h5',
 'vor0_5.h5',
 'vor5_7.h5',
 'vor4_27.h5',
 'vor5_9.h5',
 'vor9_5.h5',
 'vor4_9.h5',
 'vor0_27.h5',
 'vor5_23.h5',
 'vor8_5.h5',
 'vor0_16.h5',
 'vor7_22.h5',
 'vor2_18.h5',
 'vor8_12.h5',
 'vor5_4.h5',
 'vor8_11.h5',
 'vor2_1.h5',
 'vor2_2.h5',
 'vor7_16.h5',
 'vor8_15.h5',
 'vor7_18.h5',
 'vor5_16.h5',
 'vor4_28.h5',
 'vor7_11.h5',
 'vor2_17.h5',
 'vor6_20.h5',
 'vor9_14.h5',
 'vor6_11.h5',
 'vor7_28.h5',
 'vor2_10.h5',
 'vor5_15.h5',
 'vor3_14.h5',
 'vor4_30.h5',
 'vor0_8.h5',
 'vor1_3.h5',
 'vor4_7.h5',
 'vor3_0.h5',
 'vor8_7.h5',
 'vor7_26.h5',
 'vor3_5.h5',
 'vor6_8.h5',
 'v

In [31]:
ds_remove_all = {}
ds_all = os.listdir(dir_ds)
for key_subwork in key_subworks:
    print(key_subwork)
    filename = [file for file in ds_all if key_subwork in file][0]
    ds = h5py.File(dir_ds+filename, 'r')
    ds_remove = ds['ds_remove'][0]
    ds.close()
    ds_remove_all[key_subwork] = ds_remove

vor0_0
vor0_1
vor0_2
vor0_3
vor0_4
vor0_5
vor0_6
vor0_7
vor0_8
vor0_9
vor0_10
vor0_11
vor0_12
vor0_13
vor0_14
vor0_15
vor0_16
vor0_17
vor0_18
vor0_19
vor0_20
vor0_21
vor0_22
vor0_23
vor0_24
vor0_25
vor0_26
vor0_27
vor0_28
vor0_29
vor1_0
vor1_1
vor1_2
vor1_3
vor1_4
vor1_5
vor1_6
vor1_7
vor1_8
vor1_9
vor1_10
vor1_11
vor1_12
vor1_13
vor1_14
vor1_15
vor1_16
vor1_17
vor1_18
vor1_19


vor1_20
vor1_21
vor1_22
vor1_23
vor1_24
vor2_0
vor2_1
vor2_2
vor2_3
vor2_4
vor2_5
vor2_6
vor2_7
vor2_8
vor2_9
vor2_10
vor2_11
vor2_12
vor2_13
vor2_14
vor2_15
vor2_16
vor2_17
vor2_18
vor2_19
vor2_20
vor2_21
vor2_22
vor2_23
vor3_0
vor3_1
vor3_2
vor3_3
vor3_4
vor3_5
vor3_6
vor3_7
vor3_8
vor3_9
vor3_10
vor3_11
vor3_12
vor3_13
vor3_14
vor3_15
vor3_16
vor3_17
vor3_18
vor3_19
vor3_20
vor3_21
vor4_0
vor4_1
vor4_2
vor4_3
vor4_4
vor4_5
vor4_6
vor4_7
vor4_8
vor4_9
vor4_10
vor4_11
vor4_12
vor4_13
vor4_14
vor4_15
vor4_16
vor4_17
vor4_18
vor4_19
vor4_20
vor4_21
vor4_22
vor4_23
vor4_24
vor4_25
vor4_26
vor4_27
vor4_28
vor4_29
vor4_30
vor4_31
vor4_32
vor5_0
vor5_1
vor5_2
vor5_3
vor5_4
vor5_5
vor5_6
vor5_7
vor5_8
vor5_9
vor5_10
vor5_11
vor5_12
vor5_13
vor5_14
vor5_15
vor5_16
vor5_17
vor5_18
vor5_19
vor5_20
vor5_21
vor5_22
vor5_23
vor5_24
vor5_25
vor5_26
vor6_0
vor6_1
vor6_2
vor6_3
vor6_4
vor6_5
vor6_6
vor6_7
vor6_8
vor6_9
vor6_10
vor6_11
vor6_12
vor6_13
vor6_14
vor6_15
vor6_16
vor6_17
vor6_18
vor6_19
vo

### Pick probes' Phase velocity automatically

In [32]:
def cluster_fundamental(ds):
    global f
    global c
    global f_new
    global c_new
    global dir_ds
    global threshold
    # 归一化
    #ds = ds / np.max(ds)  
    ds = np.nan_to_num(ds)
    for i in range(ds.shape[1]):
        ds[:,i] = ds[:,i]/np.max(ds[:,i])

    binary_ds = ds > threshold
    label_ds,num_labels = measure.label(binary_ds, connectivity=2, return_num=True)
    region_areas = [np.sum(label_ds==i) for i in range(1,num_labels+1)]
    if num_labels == 0:
        return np.zeros(0),np.zeros(0)
    max_area_label = np.argmax(region_areas) + 1
    largest_cluster = label_ds == max_area_label
    indices = np.argwhere(largest_cluster)

    ff = f_new[indices[:,1]]
    cc = c_new[indices[:,0]]

    return ff,cc

In [33]:
def pick_fundamental(key):
    global f
    global c
    global f_new
    global c_new
    global flag_f
    global flag_c
    global dir_ds
    global threshold
    global dir_disp
    global dir_ds
    global info_basic
    global info_basic_bi
    global smooth_cluster
    global smooth_pick
    global key_subworks
    global ds_remove_all

    
    ds_remove = ds_remove_all[key]
    grace = 0.0

    f_start = flag_f[0]
    f_end = flag_f[1]
    c_min = flag_c[0]
    c_max = flag_c[1]

    ff_all = []
    cc_all = []


    while f_start != f_end:
        #print(f_start,f_end)
        f_new = f[np.logical_and(f>f_start,f <= f_end)]
        c_new = c[np.logical_and(c>c_min,c < c_max)]
        ds_new0 = ds_remove[np.logical_and(c>c_min,c < c_max),:]
        ds_new = ds_new0[:,np.logical_and(f>f_start,f <= f_end)]
        # 对数组进行横向的平滑
        ds_new = pd.DataFrame(ds_new)
        #ds_new = ds_new.rolling(100).mean()
        ds_new = ds_new.T
        ds_new = ds_new.rolling(smooth_cluster).mean()
        ds_new = np.array(ds_new)
        ds_new = ds_new.T
        ff,cc = cluster_fundamental(ds_new)
        if len(ff_all) == 0:
            ff_all = ff
            cc_all = cc
        elif len(ff) != 0:
            ff_all = np.hstack((ff_all,ff))
            cc_all = np.hstack((cc_all,cc))
        else:
            #print('this')
            break
        f_start = np.max(ff)
        c_max = np.max(cc[ff==f_start]) + grace
    
    # smooth
    # 转置
    """
    ds_smooth = ds_remove.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    ds_smooth = ds_smooth.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    """
    ds_smooth = plotlib.smooth_ds(ds_remove,level_f = smooth_pick,level_c = smooth_pick)

    f_range = set(ff_all)
    f_range = np.array(list(f_range))
    c_pick = []
    for i in range(len(f_range)):
        f_this = f_range[i]
        c_this = cc_all[ff_all==f_this]
        c_range = [np.min(c_this),np.max(c_this)]
        c_new = c[np.logical_and(c>=c_range[0],c<=c_range[1])]
        c_pick.append(c_new[np.argmax(ds_smooth[np.logical_and(c>=c_range[0],c<=c_range[1]),f==f_this])])
    c_pick = np.array(c_pick)
    # smooth
    index = np.argsort(f_range)
    f_range = f_range[index]
    c_pick = c_pick[index]
    c_pick_smooth = sn.gaussian_filter1d(c_pick,6)
    
    # save as txt
    np.savetxt(dir_disp+'autopick_cluster_'+key+'.txt', np.array([ff_all,cc_all]).T, fmt='%.4f', delimiter=',')
    np.savetxt(dir_disp+'autopick_'+key+'.txt', np.array([f_range,c_pick,c_pick_smooth]).T, fmt='%.4f', delimiter=',')
    print('finish '+ key + ' '+str(key_subworks.index(key)+1) + '/'+str(len(key_subworks)))

In [34]:
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
flag_c = [0.25,1.2]
flag_f = [5,15]
#flag_f = [6,25]
threshold = 0.9
f_new = f[np.logical_and(f>flag_f[0],f < flag_f[1])]
c_new = c[np.logical_and(c>flag_c[0],c < flag_c[1])]
smooth_cluster = 30
smooth_pick = 3
flag_pick = 1

In [35]:
warnings.filterwarnings("ignore")
if flag_pick == 1:
    for key_subwork in key_subworks:
        pick_fundamental(key_subwork)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
def plot_disp(key_subwork):
    global dir_disp
    global f
    global c
    global dir_image
    global key_subworks

    fig,ax = plt.subplots(1,3,figsize=(18,6))
    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]

    title0 = key_subwork
    ff_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    c_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    c_smooth = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,2]

    ax[0] = plotlib.plot_fj(ax[0],ds_remove,title0,f,c,0,c_map='jet')
    ax[0].scatter(ff_all,cc_all,marker='o',s=10,color='k')
    
    ds_smooth = plotlib.smooth_ds(ds_remove,level_f = 10,level_c=6)


    ax[1] = plotlib.plot_fj(ax[1],ds_smooth,title0,f,c,0,c_map='jet')
    ax[1].scatter(f_pick,c_pick,marker='o',s=10,color='k')

    ax[2] = plotlib.plot_fj(ax[2],ds_smooth,title0,f,c,0,c_map='jet')
    ax[2].scatter(f_pick,c_smooth,marker='o',s=10,color='k')

    plt.savefig(dir_image+'autopick_'+key_subwork+'.png',dpi=60)
    plt.tight_layout()
    plt.close()
    print('finish '+key_subwork,' ', str(key_subworks.index(key_subwork)+1),'/',str(len(key_subworks)))

In [ ]:
flag_plot = 1

In [ ]:
if flag_plot == 1:
    for key_subwork in key_subworks:
        plot_disp(key_subwork)